# Initialization
Note that, as in the Titanic assignment, the housing pricees dataset is locally stored in the folder structure described in the repository Readme.

## Environmental setup

In [ ]:
# environmental setup
import os;  import sys;  import numpy as np;  import pandas as pd;  import sklearn

# allow inline display of matplotlib graphs
import matplotlib as mpl;   import matplotlib.pyplot as plt
%matplotlib inline

# suppress errors, as seen in Lecture 8; see SciPy issue #5998
import warnings;   warnings.filterwarnings(action="ignore", message="^internal gelsd")

# unify RNG across runs
seedNum=0;  np.random.seed(seedNum)

# read content of training + test datasets
dsamp=pd.read_csv('../Datasets/housePrices/train.csv')

# make entry ID into dataframe index
dsamp.set_index('Id', inplace=True)

print('Data/library loading complete!')

## Data (re-)organization
The dataset contains a large number of variables, some of which are redundant or may be relevant in multiple aspects. Considering that the Amereican housing market is often valued on the availability of specific amenities, several categories were defined, and relevant variables were grouped under them. Principal component analysis will be conducted on each category to reduce the dimensionality of the final input into the eventual ML model.

To allow this sort of analysis, a function will, first, be created to easily normalize columns in a dataframe. Since it is known that many variables are non-normal, capabilities to automatically apply log-transforms and the Wilson-Hilferty transform for Rayleigh-distributed data were also included.

In [ ]:
import scipy.stats as stats

def normalize(df,a):
    for col in range(len(df.columns)):
        dfNow=pd.DataFrame( df.iloc[:,col], columns=[df.columns[col]] )
        x=dfNow.values.flatten()         # SciPy requires input as 1-D array
        z1,p1=stats.normaltest(x)
        if p1<a and isinstance(x,float): # attempt 1: log-transform
            x=np.log(x+abs(min(x))+1)
            z2,p2=stats.normaltest(x)
            if p2<a:                     # attempt 2: Wilson-Hilferty transform for Rayleigh distr.
                x=x**(2/3)
        df.iloc[:,col]=x
    return pd.DataFrame(df, index=df.index, columns=df.columns)

alpha=0.01;   

### Geography
First, variables pertaining to the geography of each listing were extracted and modified. This includes information about both the property in its local environment (e.g. terrain, adjacent roads) as well as its context in Ames, IA as a whole.

| Variable Name | Variable Meaning             | Type    | Discard? | Convert to: |
| :---          | :---                         | :---    | :---     | :---        |
| MSZoning      | Zoning classification        | object  | Partial  | see note 01 |
| LotFrontage   | Length of curb @ prop. (ft)  | object  |          | Integer     |
| Street        | Road access type             | object  | YES      |             |
| Alley         | Alley access type            | object  | YES      |             |
| LotShape      | Property shape irregularity  | object  |          | Ordinal     |
| LandContour   | Property flatness            | object  |          | 1-hot enc.  |
| LotConfig     | Lot location wrt streetfront | object  |          | 1-hot enc.  |
| LandSlope     | Slope of property            | object  |          | Ordinal     |
| Neighborhood  | Neighborhood in Ames, IA     | object  |          | Ordinal     |
| Condition1    | Proximity to various places  | object  | Partial  | see note 02 |
| Condition2    | - see above -                | object  | Partial  | see note 02 |
| YearBuilt     | Year of original construc.   | int64   |          |             |

The street, alley categories were removed, as those categories were nearly homogeneous in the training dataset.

The neighborhood location was converted into ordinal values based on each neighborhood's median home value.

**Note 01:** Zoning classifications were transformed differently for each variable. Categories that are not formally residential -Agriculture, Commercial, and Industrial- were deleted. The remaining "Residential" zones were ranked ordinally, with "Low Density Park" being ranked in between "Low" and "Medium" densities and "Floating Village Residential" being treated below "Low" density rank.

**Note 02:** Conditions of a property are ordinally encoded based on the following categories:

* Automobile Traffic (Artery > Feedr > n/a)
* East-West Rail (RRAe > RRNe > n/a)
* North-South Rail (RRAn > RRNn > n/a)
* "Positive" Off-site Feature (PosA > PosN > n/a)

"Norm" was eliminated due to redundancy. Note that the distinction between the North-South and East-West rails were preserved. This is because the track usage for these two rail rights-of-way may be significantly different, considering significant differences in how they contribute to rail owner Union Pacific's strategy (also see https://iowadot.gov/iowainmotion/railplan/2017/IowaSRP2017_AppendixA.pdf).

In [ ]:
areas=dsamp['Neighborhood'].unique()                # list of neighborhoods
subdivID=range(len(areas))                          # get IDs for neighborhoods

medians=np.ndarray(len(areas),)
for n in range(len(areas)):
    x=dsamp['Neighborhood'].str.contains(areas[n])  # "dsamp" entries for current subdivision
    medians[n]=np.median( dsamp.loc[x, 'SalePrice'].to_numpy() ) # median home sale price

medseq=[x for x,y in sorted(enumerate(medians),     # index of neighborhood labels, in order of
                            key = lambda x: x[1])]  # increasing median house market value

fig=plt.figure(figsize=(12, 6))                     # initialize figure
ax=fig.add_axes([0,0,1,1])
xPos=0
for n in medseq:   
    x=dsamp['Neighborhood'].str.contains(areas[n])  # "dsamp" entries for current subdivision
    X=dsamp.loc[x, 'SalePrice'].to_numpy()          # all home sale price in above subdiv.
    vp=ax.violinplot(X, [xPos], points=80, vert=True, widths=0.7, showextrema=True, showmedians=True)
    xPos=xPos+1                                     # manual change in plot position


plt.xticks(range(len(areas)), areas, rotation='vertical')
plt.grid(color='k', linewidth=1, alpha=0.05)
plt.show()

def org_Neighborhood(df_in):
    df_in=df_in[['Neighborhood']]
    return pd.get_dummies(df_in, prefix="nghb", dtype="int64")

In [ ]:
def org_MSZoning(df_in):
    df_in=df_in[['MSZoning']]
    df_zones=pd.get_dummies(df_in, prefix="zone", dtype="int64")
    if 'zone_FV' not in df_zones: df_zones['zone_FV']=0 # contingency, float. vill.
    if 'zone_RP' not in df_zones: df_zones['zone_RP']=0 # contingency, res.-park
    df_zones['zone_res']=\
     df_zones['zone_FV'] + 2*df_zones['zone_RP'] +\
     3*df_zones['zone_RM'] + 4*df_zones['zone_RH']  # ordinal encoding "residential"s
    return df_zones[['zone_res']]

def org_LotFrontage(df_in):
    df_in=df_in[['LotFrontage']]
    cleanNaN=lambda x: 0.0 if (x is None or np.isnan(x)) else float(x)
    df_out=df_in['LotFrontage'].apply(cleanNaN)
    return df_out.to_frame(name='frntg')

def org_LotShape(df_in):
    df_in=df_in[['LotShape']]
    irreg={'Reg':0, 'IR1':1, 'IR2':2, 'IR3':3}
    return normalize( df_in['LotShape'].replace(irreg).to_frame(name='irreg').astype(int), alpha)

def org_LandContour(df_in):
    df_in=df_in[['LandContour']]
    return pd.get_dummies(df_in, prefix="ctur", dtype="int64")

def org_LotConfig(df_in):
    df_in=df_in[['LotConfig']]
    return pd.get_dummies(df_in, prefix="conf", dtype="int64")

def org_LandSlope(df_in):
    df_in=df_in[['LandSlope']]
    irreg={'Gtl':0, 'Mod':1, 'Sev':2}
    return normalize( df_in['LandSlope'].replace(irreg).to_frame(name='slope').astype(int), alpha)

def org_Condition(df_in):
    df1=df_in[['Condition1']];  df2=df_in[['Condition2']]
    df1=pd.get_dummies(df1, prefix="cond", dtype="int64")  # 1-hot, condition 1
    df2=pd.get_dummies(df2, prefix="cond", dtype="int64")  # 1-hot, condition 2
    df1.drop(columns=['cond_Norm']);   df2.drop(columns=['cond_Norm'])
    conds=['cond_Artery', 'cond_Feedr', 'cond_RRNn',   'cond_RRAn',\
           'cond_RRNe',   'cond_RRAe',  'cond_PosN',   'cond_PosA']
    for cond in conds:
        if cond not in df1: df1[cond]=0
        if cond not in df2: df2[cond]=0
    df_all=df1+df2
    df_all['cond_traf']=df_all['cond_Feedr']+ 2*df_all['cond_Artery']
    df_all['cond_RRns']=df_all['cond_RRNn'] + 2*df_all['cond_RRAn']
    df_all['cond_RRew']=df_all['cond_RRNe'] + 2*df_all['cond_RRAe']
    df_all['cond_qol']= df_all['cond_PosN'] + 2*df_all['cond_PosA']
    return df_all[['cond_traf','cond_RRns','cond_RRew','cond_qol']].astype(int)

def org_geography(df_in):
    df=pd.concat([org_MSZoning(df_in),\
                  org_LotConfig(df_in),org_LotShape(df_in),\
                  org_LotFrontage(df_in),org_LotShape(df_in),\
                  org_LandContour(df_in),org_LandSlope(df_in),\
                  org_Neighborhood(df_in),org_Condition(df_in),\
                  df_in[['YearBuilt']]\
                 ], axis=1)
    return df

### Structure
Next, variables concerning the structure of the houses were isolated.

#### General
The following variables, all of which encode information about the general structure of the property in question, were modified together.

| Variable Name | Variable Meaning             | Type    | Discard? | Convert to: |
| :---          | :---                         | :---    | :---     | :---        |
| YearBuilt     | Year of original construc.   | int64   |          |             |
| YearRemodAdd  | Remodel yr. (=built if none) | int64   |          | see note 03 |
| LotArea       | Proprety area (sq ft)        | int64   |          |             |
| BldgType      | Type of dwelling             | object  |          | 1-hot enc.  |
| MSSubClass    | Type of dwelling             | int64   | Partial  | see note 04 |
| HouseStyle    | Style of dwelling (# floor)  | object  |          | see note 05 |
| OverallQual   | Rating of material/quality   | int64   |          |             |
| OverallCond   | Rating of house condition    | int64   |          |             |
| Foundation    | Foundation type              | object  |          | 1-hot enc.  |

**Note 03:** Remodeling years will be preserved as integers, but the difference between original and remodeled years will be taken to imply whether a house was rebuilt at all.

**Note 04:** Class 190 (2-family conversion) and 120-180 (Planned Unit Development) properties will be one-hot encoded. Other subclasses will be eliminated, as they are redundant to other variables.

**Note 05:** Floors (1, 1.5, 2, or 2.5) will be stored as ordinal ranks (1 to 4), with "Split Foyer" being classified as a 1.5-floor property and "Split Level" houses being treeated as a two-floor property. Finished and unfinished floors are not distinguished here.

In [ ]:
def org_years(df_in):
    df_ori=df_in[['YearBuilt']]
    df_mod=df_in[['YearRemodAdd']] - df_ori.values
    return pd.concat([df_ori, df_mod], axis=1)

def org_BldgType(df_in):
    df_in=df_in[['BldgType']]
    return pd.get_dummies(df_in, prefix="type", dtype="int64")

def org_MSSubClass(df_in):
    df_in=df_in[['MSSubClass']].copy()    # explicitly allocate memory
    cond1=df_in.MSSubClass < 120          # combine subclasses into categories
    cond2=( df_in['MSSubClass'] >=120 )&\
          ( df_in['MSSubClass'] < 190 )
    cond3=df_in.MSSubClass==190
    df_in.loc[cond1, 'MSSubClass']='std'  # make groups into strings
    df_in.loc[cond2, 'MSSubClass']='PUD'
    df_in.loc[cond3, 'MSSubClass']='2FC'
    df_out=pd.get_dummies(df_in, prefix="subc", dtype="int64")
    return df_out.drop(['subc_std'], axis=1).astype(int)

def HouseStyle(df_in):
    df_in=df_in[['HouseStyle']]
    df_zones=pd.get_dummies(df_in, prefix="styl", dtype="int64")
    conds=['styl_1Story', 'styl_1.5Fin', 'styl_1.5Unf',\
           'styl_2Story', 'styl_2.5Fin', 'styl_2.5Unf',\
           'styl_SFoyer', 'styl_SLvl']
    resps=[1, 2, 2, 3, 4, 4, 2, 3]        # indicators corresponding to "conds"
    for cond in np.arange(len(conds)):
        if conds[cond] in df_zones:
            df_zones.loc[ df_zones[conds[cond]]==1, 'Style']=resps[cond]
        else:
            df_zones['Style']=0
    return df_zones[['Style']].astype(int)

def org_Foundation(df_in):
    df_in=df_in[['Foundation']]
    return pd.get_dummies(df_in, prefix="fndt", dtype="int64")

def org_strGen(df_in):
    df=pd.concat([org_years(df_in),df_in[['LotArea']],\
                  org_MSSubClass(df_in),HouseStyle(df_in),\
                  df_in[['OverallQual','OverallCond']],\
                  org_Foundation(df_in)\
                 ], axis=1)
    return df

#### Exterior
Next, information on the house's exterior were modified as follows.

| Variable Name | Variable Meaning             | Type    | Discard? | Convert to: |
| :---          | :---                         | :---    | :---     | :---        |
| RoofStyle     | Roof geometry                | object  |          | 1-hot enc.  |
| RoofMatl      | Roof material                | object  | YES      |             |
| Exterior1st   | Exterior material            | object  |          | 1-hot enc.  |
| Exterior2nd   | - see above -                | object  |          | 1-hot enc.  |
| MasVnrType    | Masonry veneer type          | object  | YES      | see note 06 |
| MasVnrArea    | Masonry veneer area (sq ft)  | object  |          | see note 06 |
| ExterQual     | Exterior material quality    | object  |          | Ordinal     |
| ExterCond     | Exterior condition           | object  |          | Ordinal     |

The roof material category was removed, as the vast majority of the training dataset belonged in the composite shingles group.

**Note 06:** Masonry veneer area will be encoded as their own integers, separated by masonry type.

In [ ]:
def org_RoofStyle(df_in):
    df_in=df_in[['RoofStyle']]
    return pd.get_dummies(df_in, prefix="rstyle", dtype="int64")

def org_Exterior(df_in):
    df1=df_in[['Exterior1st']].copy()                     # explicitly allocate memory
    df2=df_in[['Exterior2nd']].copy()
    df2[ df2=='Brk Cmn' ]='BrkComm'                       # correct inconsistent labels
    df2[ df2=='CmentBd' ]='CemntBd'
    df2[ df2=='Wd Shng' ]='WdShing'
    df1=pd.get_dummies(df1, prefix="ext", dtype="int64")  # 1-hot encode each mat type
    df2=pd.get_dummies(df2, prefix="ext", dtype="int64")
    dfAll=df1.add(df2, axis=1, fill_value=0)
    conds=['ext_AsbShng', 'ext_AsphShn', 'ext_BrkComm','ext_BrkFace', 'ext_CBlock',\
           'ext_CemntBd','ext_HdBoard', 'ext_ImStucc','ext_MetalSd', 'ext_Other',\
           'ext_Plywood', 'ext_PreCast', 'ext_Stone',  'ext_Stucco',  'ext_VinylSd',\
           'ext_Wd Sdng', 'ext_WdShing']
    for cond in conds:
        if cond not in dfAll: dfAll[cond]=0
    return dfAll.astype(int)
    
def org_masonry(df_in):
    df_type=pd.get_dummies(df_in[['MasVnrType']], prefix="masstyle", dtype="int64")
    df_type.drop(labels='masstyle_None', axis=1, inplace=True)
    df_area=df_in[['MasVnrArea']]
    df_area.fillna(value=0, inplace=True)
    return pd.DataFrame(df_area.values*df_type.values,\
                        columns=df_type.columns, index=df_area.index, dtype=int)

def org_ExterQual(df_in):
    df_in=df_in[['ExterQual']]
    exqu={'Po':-2, 'Fa':-1, 'TA':0, 'Gd':1, 'Ex':2}
    return df_in['ExterQual'].replace(exqu).to_frame(name='extQual').astype(int)

def org_ExterCond(df_in):
    df_in=df_in[['ExterCond']]
    exco={'Po':-2, 'Fa':-1, 'TA':0, 'Gd':1, 'Ex':2}
    return df_in['ExterCond'].replace(exco).to_frame(name='extCond').astype(int)

def org_exteriors(df_in):
    df=pd.concat([org_RoofStyle(df_in),\
                  org_Exterior(df_in),  org_masonry(df_in),\
                  org_ExterQual(df_in), org_ExterCond(df_in)\
                 ], axis=1)
    return df

#### Basement
Likewise, information on a property's basement was encoded as follows. Note that properties without a basement were assigned zero or median values as appropriate.

| Variable Name | Variable Meaning             | Type    | Discard? | Convert to: |
| :---          | :---                         | :---    | :---     | :---        |
| BsmtQual      | Basement height, categorized | object  |          | see note 07 |
| BsmtCond      | Basement condition           | object  |          | Ordinal     |
| BsmtExposure  | Walkout/garden-level walls?  | object  |          | 1-hot enc.  |
| BsmtFinType1  | Rating, finished basement    | object  |          | see note 08 |
| BsmtFinSF1    | above, square footage        | int64   | YES      | see note 09 |
| BsmtFinType2  | ditto, multiple types        | object  |          | see note 08 |
| BsmtFinSF2    | see above                    | int64   | YES      | see note 09 |
| BsmtUnfSF     | Unfinished basement sq ft    | int64   |          |             |
| TotalBsmtSF   | Total basement sq ft         | int64   | YES      |             |
| BsmtFullBath  | Basement full bathroom       | int64   |          |             |
| BsmtHalfBath  | Basement half bathroom       | int64   |          |             |

**Note 07:** Basement height will be categorized based on whether it does NOT have a low ceiling. Properties with no basement will be classifieed as "no".

**Note 08:** One-hot encoding was applied to both of these entries, together.

**Note 09:** The square footage will bee combined with the basement finish types.

In [ ]:
def org_BsmtQual(df_in):
    df_in=df_in[['BsmtQual']]
    df_in.fillna(value='NA', inplace=True)
    bsqu={'NA':0, 'Po':0, 'Fa':0, 'TA':1, 'Gd':1, 'Ex':1}
    return df_in['BsmtQual'].replace(bsqu).to_frame(name='bsmtQual').astype(int)

def org_BsmtCond(df_in):
    df_in=df_in[['BsmtCond']]
    df_in.fillna(value='NA', inplace=True)
    bsco={'NA':0, 'Po':-2, 'Fa':-1, 'TA':0, 'Gd':1, 'Ex':2}
    return df_in['BsmtCond'].replace(bsco).to_frame(name='bsmtCond').astype(int)

def org_BsmtExposure(df_in):
    df_in=df_in[['BsmtExposure']]
    df_in.fillna(value='NA', inplace=True)
    bsex={'NA':0, 'No':0, 'Mn':1, 'Av':2, 'Gd':3}
    return df_in['BsmtExposure'].replace(bsex).to_frame(name='bsmtExpo').astype(int)

def org_BsmtFinType(df_in):
    dfT1=pd.get_dummies(df_in[['BsmtFinType1']], prefix="bsfin", dtype="int64")
    dfT2=pd.get_dummies(df_in[['BsmtFinType2']], prefix="bsfin", dtype="int64")
    dfF1=pd.get_dummies(df_in[['BsmtFinSF1']],   prefix="bs_sf", dtype="int64")
    dfF2=pd.get_dummies(df_in[['BsmtFinSF2']],   prefix="bs_sf", dtype="int64")
    dfT1.fillna(value=0, inplace=True)
    dfT2.fillna(value=0, inplace=True)
    dfF1.fillna(value=0, inplace=True)
    dfF2.fillna(value=0, inplace=True)
    df1=pd.DataFrame( dfT1.values * dfF1.values,\
                      columns=dfT1.columns, index=dfT1.index, dtype=int)
    df2=pd.DataFrame( dfT2.values * dfF2.values,\
                      columns=dfT2.columns, index=dfT2.index, dtype=int)
    return pd.DataFrame( df1.values + df2.values,\
                      columns=df1.columns, index=df1.index, dtype=int)

def org_TotalBsmtSF(df_in):
    df_in=df_in[['TotalBsmtSF']]
    df_in.fillna(value=0, inplace=True)
    return df_in.astype(int)

def org_BsmtBaths(df_in):
    df_in=df_in[['BsmtFullBath','BsmtHalfBath']]
    df_in.fillna(value=0, inplace=True)
    return df_in.astype(int)

def org_basement(df_in):
    df=pd.concat([org_BsmtQual(df_in),     org_BsmtCond(df_in),\
                  org_BsmtExposure(df_in), org_BsmtFinType(df_in),\
                  org_TotalBsmtSF(df_in),  org_BsmtBaths(df_in)\
                 ], axis=1)
    return df

#### HVAC and Utilities
Information on a property's heating, ventilation, air conditioning, electrical systems, and fireplacs were also considered and modified as follows.

| Variable Name | Variable Meaning             | Type    | Discard? | Convert to: |
| :---          | :---                         | :---    | :---     | :---        |
| Utilities     | Utility types missing        | object  | YES      |             |
| Heating       | Heating type                 | object  | YES      |             |
| HeatingQC     | Heating quality/condition    | object  |          | Ordinal     |
| CentralAir    | Central AC?                  | object  |          | int64       |
| Electrical    | Electrical system quality    | object  |          | see note 10 |
| Fireplaces    | Fireplace count              | int64   |          |             |
| FireplaceQu   | Fireplace quality            | object  |          | Ordinal     |

While the Utilities category should, ideally, be a critical indicator for the value of a property listing, nearly all datapoints in the training dataset contained properties with full access to public utilities. Thus, this category was omitted from analysis.

**Note 10:** The quality of the electrical system will be listed ordinally, as described in the data description file. A "mixed" entry will be treated identically to a "FuseA" class.

In [ ]:
def org_HeatingQC(df_in):
    df_in=df_in[['HeatingQC']]
    hequ={'Po':-2, 'Fa':-1, 'TA':0, 'Gd':1, 'Ex':2}
    return df_in['HeatingQC'].replace(hequ).to_frame(name='HeatQual').astype(int)

def org_CentralAir(df_in):
    df_in=df_in[['CentralAir']]
    pool={'N':0, 'Y':1}
    return df_in['CentralAir'].replace(pool).to_frame(name='centAir').astype(int)

def org_Electrical(df_in):
    df_in=df_in[['Electrical']]
    df_in.fillna(value='SBrkr', inplace=True)
    elec={'SBrkr':0, 'FuseA':1, 'FuseF':2, 'FuseP':3, 'Mix':1}
    return df_in['Electrical'].replace(elec).to_frame(name='electr').astype(int)

def org_FireplaceQu(df_in):
    df_in=df_in[['FireplaceQu']]
    df_in.fillna(value='NA', inplace=True)
    fpqu={'NA':0, 'Po':-2, 'Fa':-1, 'TA':0, 'Gd':1, 'Ex':2}
    return df_in['FireplaceQu'].replace(fpqu).to_frame(name='FireQual').astype(int)

def org_util(df_in):
    df=pd.concat([org_HeatingQC(df_in),\
                  org_CentralAir(df_in), org_Electrical(df_in),\
                  df_in[['Fireplaces']], org_FireplaceQu(df_in)\
                 ], axis=1)
    return df

#### Above-Grade Internals
In contrast with below-grade (below-surface level) structures in the Basement section, above-ground internal features of a property listing are listed here.

| Variable Name | Variable Meaning             | Type    | Discard? | Convert to: |
| :---          | :---                         | :---    | :---     | :---        |
| 1stFlrSF      | 1F square footage            | int64   |          |             |
| 2ndFlrSF      | 2F square footage            | int64   |          |             |
| LowQualFinSF  | Lo-qual. finished sq ft      | int64   | YES      |             |
| GrLivArea     | Above-grade living area sqft | int64   |          |             |
| FullBath      | Above-grade full bathroom    | int64   |          |             |
| HalfBath      | Above-grade half bathroom    | int64   |          |             |
| BedroomAbvGr  | Above-grade bedrooms         | int64   |          |             |
| KitchenAbvGr  | Above-grade kitchen          | int64   |          |             |
| KitchenQual   | Kitchen quality              | object  |          | Ordinal     |
| TotRmsAbvGrd  | Above-grade rooms            | int64   | YES      |             |

In [ ]:
def org_KitchenQual(df_in):
    df_in=df_in[['KitchenQual']]
    df_in.fillna(value='TA', inplace=True)
    kiqu={'Po':-2, 'Fa':-1, 'TA':0, 'Gd':1, 'Ex':2}
    return df_in['KitchenQual'].replace(kiqu).to_frame(name='KitcQual').astype(int)

def org_above(df_in):
    df=pd.concat([df_in[['1stFlrSF','2ndFlrSF','GrLivArea',\
                         'FullBath','HalfBath','BedroomAbvGr',\
                         'KitchenAbvGr']],\
                  org_KitchenQual(df_in)\
                 ], axis=1)
    return df

#### Garage
Information on the garage (if present; otherwise zero or median-impute) were modified as follows.

| Variable Name | Variable Meaning             | Type    | Discard? | Convert to: |
| :---          | :---                         | :---    | :---     | :---        |
| GarageType    | Garage location              | object  |          | 1-hot enc.  |
| GarageYrBlt   | Year of garage completion    | int64   |          |             |
| GarageFinish  | Interior finish of garage    | object  |          | ordinal     |
| GarageCars    | Max num cars in garage       | int64   |          |             |
| GarageArea    | Garage sq ft                 | int64   | YES      |             |
| GarageQual    | Garage quality               | object  |          | ordinal     |
| GarageCond    | Garage condition             | object  |          | ordinal     |
| PavedDrive    | Paved driveway?              | object  |          | ordinal     |


In [ ]:
def org_GarageType(df_in):
    df_in=df_in[['GarageType']]
    return pd.get_dummies(df_in, prefix="gtype", dtype="int64")

def org_GarageYrBlt(df_in):
    df_in=df_in[['GarageYrBlt']]
    df_in['GarageYrBlt'].fillna(1965, inplace=True) # impute w/ approximate mode
    return df_in[['GarageYrBlt']].astype(float)

def org_GarageCars(df_in):
    df_in=df_in[['GarageCars']]
    df_in.replace(to_replace=[None], value=0, inplace=True)#contingency
    df_in.fillna(value=0, inplace=True)
    return df_in.astype(int)

def org_GarageFinish(df_in):
    df_in=df_in[['GarageFinish']]
    df_in['GarageFinish'].fillna('NA', inplace=True)
    pool={'NA':0, 'Unf':1, 'RFn':2, 'Fin':3}
    return df_in['GarageFinish'].replace(pool).to_frame(name='gfin').astype(int)

def org_GarageQual(df_in):
    df_in=df_in[['GarageQual']]
    df_in['GarageQual'].fillna('NA', inplace=True)
    pool={'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}
    return df_in['GarageQual'].replace(pool).to_frame(name='gqual').astype(int)

def org_GarageCond(df_in):
    df_in=df_in[['GarageCond']]
    df_in['GarageCond'].fillna('NA', inplace=True)
    pool={'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}
    return df_in['GarageCond'].replace(pool).to_frame(name='gcond').astype(int)

def org_PavedDrive(df_in):
    df_in=df_in[['PavedDrive']]
    df_in['PavedDrive'].fillna('NA', inplace=True)
    pool={'N':0, 'P':1, 'Y':2}
    return df_in['PavedDrive'].replace(pool).to_frame(name='gpave').astype(int)

def org_garage(df_in):
    df=pd.concat([org_GarageType(df_in),org_GarageYrBlt(df_in),org_GarageCars(df_in),\
                  org_GarageFinish(df_in), org_GarageQual(df_in),\
                  org_GarageCond(df_in), org_PavedDrive(df_in)\
                 ], axis=1)
    return df

#### Porch/Yard Enhancements
Properties with a porch or yard, as well as relevant features, were modified as follows.

| Variable Name | Variable Meaning             | Type    | Discard? | Convert to: |
| :---          | :---                         | :---    | :---     | :---        |
| WoodDeckSF    | Wood deck area, sq ft        | int64   |          |             |
| OpenPorchSF   | Open porch area, sq ft       | int64   |          |             |
| EnclosedPorch | Enclosed porch area, sq ft   | int64   |          |             |
| 3SsnPorch     | 3-screen porch area, sq ft   | int64   |          |             |
| ScreenPorch   | Screened porch area, sq ft   | int64   |          |             |
| PoolArea      | Pool area, sq ft             | int64   |          |             |
| PoolQC        | Pool quality                 | object  | YES      |             |
| Fence         | Fence quality                | object  | YES      |             |

In [ ]:
def org_yard(df_in):
    df=df_in[['WoodDeckSF','OpenPorchSF','EnclosedPorch',\
              '3SsnPorch','ScreenPorch','PoolArea']]
    return df

#### Other Features
Extra or luxury features that enhance quality of life beyond the expectations of a normal house are categorized here.

| Variable Name | Variable Meaning             | Type    | Discard? | Convert to: |
| :---          | :---                         | :---    | :---     | :---        |
| MiscFeature   | Other features               | object  | YES      |             |
| MiscVal       | Dollar value of above        | object  | see note | int64       |

**Note 11:** The dollar value of miscellaneous features will be removed from the analysis. Instead, its valuation will be added directly onto the predicted property value.

In [ ]:
def org_separateMisc(df_in):
    dfo=df_in[['SalePrice']].copy()
    dfi=df_in[['MiscVal']].copy()
    df=pd.concat([df_in.drop(labels='SalePrice', axis=1, inplace=True),\
                  pd.DataFrame(dfo.values - dfi.values, \
                               columns=['BaseVal'], index=dfo.index, dtype=float)\
                 ], axis=1)
    return df

def org_considerMisc(df_base, df_misc):
    dfo=df_base[['BaseVal']].copy()
    df_misc=df_misc[['MiscVal']].copy()
    df=pd.DataFrame( dfo.values + df_misc.values, \
                     columns=['SalePrice'], index=dfo.index, dtype=float)
    return df

### Sales Condition
Finally, information that considers the transaction of a property, as well as liabilities that directly affect it, are listed.

#### Sales
Basic information on how and when a property is sold is encoded here.

| Variable Name | Variable Meaning             | Type    | Discard? | Convert to: |
| :---          | :---                         | :---    | :---     | :---        |
| MoSold        | Month sold (MM)              | int64   |          | 1-hot enc.  |
| YrSold        | Year sold (YYYY)             | int64   |          |             |
| SaleType      | Type of sale                 | object  |          | 1-hot enc.  |
| SaleCondition | Condition of sale            | object  |          | 1-hot enc.  |

In [ ]:
def org_MoSold(df_in):
    df_in=df_in[['Foundation']]
    return pd.get_dummies(df_in, prefix="smonth", dtype="int64")

def org_SaleType(df_in):
    df_in=df_in[['SaleType']]
    return pd.get_dummies(df_in, prefix="stype", dtype="int64")

def org_SaleCondition(df_in):
    df_in=df_in[['SaleCondition']]
    return pd.get_dummies(df_in, prefix="scond", dtype="int64")

def org_sales(df_in):
    df=pd.concat([org_MoSold(df_in),\
                  normalize( df_in[['YrSold']].astype(int), alpha ),\
                  org_SaleType(df_in), org_SaleCondition(df_in)\
                 ], axis=1)
    return df

#### Faults
The following variables encode faults, liabilities, and compromised features that, intuitively, negatively affect a property's value.

| Variable Name | Variable Meaning             | Type    | Discard? | Convert to: |
| :---          | :---                         | :---    | :---     | :---        |
| OverallQual   | Rating of material/quality   | int64   |          |             |
| OverallCond   | Rating of house condition    | int64   |          |             |
| Functional    | Deductions for home function | object  |          | Ordinal     |

In [ ]:
def org_Functional(df_in):
    df_in=df_in[['Functional']]
    df_in['Functional'].fillna('Min2', inplace=True) # assume minor damages for entries w/o fault ratings
    fault={'Typ':0, 'Min1':1, 'Min2':2, 'Mod':3, 'Maj1':4, 'Maj2':5, 'Sev':6, 'Sal:':7}
    df_in=df_in['Functional'].replace(fault).to_frame(name='fault')
    return normalize( df_in, alpha )

def org_faults(df_in):
    df=pd.concat([normalize( df_in[['OverallQual','OverallCond']].astype(float), alpha),\
                  org_Functional(df_in)\
                 ], axis=1)
    return normalize(df,alpha)

### Bringing Everything Together
Now that we have (finally!!) defined the functions that will re-format our data in helpful ways, we will actually apply them to our datasets.

In [ ]:
def organize(df_in):
    return pd.concat([org_geography(df_in), org_strGen(df_in),\
                      org_exteriors(df_in), org_basement(df_in),\
                      org_util(df_in), org_above(df_in), org_garage(df_in),\
                      org_yard(df_in), org_sales(df_in), org_faults(df_in)\
                     ], axis=1)

# remove misc. values from true dollar value (log-normalized)
df=dsamp[['MiscVal', 'SalePrice']].copy()
truth=np.log( org_separateMisc(df).values.ravel() )

# organize training and testing datsets
ds=organize(dsamp)

# impose homoskedasticity
from sklearn.preprocessing import RobustScaler
scaler=RobustScaler().fit(ds.values)
ds_scaled=scaler.transform(ds.values)

# convert scaled inputs to DataFrames
ds=pd.DataFrame(ds_scaled, index=ds.index, columns=ds.columns)

print('Data organization complete!')

# Train model
Now that the training dataset has been pruned and transformed into a ML-friendly format, various learning techniques will be employed (and, eventually, the best variant of the best learning type will be identified).

To do so, the following steps will be executed:

* Determine candidate methods for dimensionality reduction using stochastic gradient descent
* Using the candidates, identify the best regression learning algorithm
* Using the best regression learner, re-run candidate manifold learners, and select the best dim. reducer
* Optimize hyperparameters using grid-based cross-validation

### Determine Dimensionality Reduction Technique
First, candidate methods to reduce the dimensionality of our dataset will be chosen.

Stochastic gradient descent will be used as the baseline model type to compare performance.

In [ ]:
# dimensionality reduction techniques
from sklearn.decomposition  import PCA, KernelPCA
from sklearn.manifold       import LocallyLinearEmbedding, MDS, SpectralEmbedding, Isomap, TSNE
from sklearn.pipeline       import Pipeline

# supervised regression techniques
from sklearn.linear_model   import SGDRegressor

# if valid, number of iterations to train per epoch + tolerance
iterVal=1000
toleVal=1e-6

The Kaggle competition uses the root-mean-squared logarithmic error to rank submissions, but this is not defined in SciKit-Learn. We will manually define this loss function, instead (but in its absolute-value form, so that it can be minimized).

In [ ]:
from sklearn.metrics.scorer  import make_scorer;   import math
from sklearn.model_selection import cross_validate

def ARMSL(truth,pred):
    SLE=0
    eps=1e-16
    for i in np.arange(len(pred)):
        if pred[i]+1 > eps:
            SLE += (np.log(pred[i]+1) - np.log(truth[i]+1))**2
        else:
            SLE += (np.log(eps) - np.log(truth[i]+1))**2    # prevent log explosion
    return abs(np.sqrt( SLE.mean() ))

armsle=make_scorer(ARMSL, greater_is_better=False)

def checkARMSLE(MLmodel, df, truth, scoreObj, cv, lim):
    scoring={'ARMSLE':scoreObj}
    scores=cross_validate(MLmodel, df, truth, scoring=scoring, cv=cv)
    arm=-scores['test_ARMSLE'].mean() # get average of abs-root-mean-squared-log errors
    armFmt=round(arm, 2)  if arm<lim  else 'above limit'
    return armFmt

We'll create a dictionary of dimensionality reduction techniques to iterate through candidates.

In [ ]:
reducers={'Regular PCA (95th %)':    PCA(random_state=seedNum, n_components=0.95),
          'Kernel PCA (p=3/n=20)':   KernelPCA(random_state=seedNum, kernel='poly',   n_components=20, degree=3, n_jobs=-1),
          'Kernel PCA (p=3/n=80)':   KernelPCA(random_state=seedNum, kernel='poly',   n_components=80,degree=3, n_jobs=-1),
          'Kernel PCA (cos/n=20)':   KernelPCA(random_state=seedNum, kernel='cosine', n_components=20, n_jobs=-1),
          'Kernel PCA (cos/n=80)':   KernelPCA(random_state=seedNum, kernel='cosine', n_components=80,n_jobs=-1),
          'Loc. Lin. Emb. (n=30)':   LocallyLinearEmbedding(random_state=seedNum, n_components=30, n_jobs=-1),
          'Loc. Lin. Emb. (n=60)':   LocallyLinearEmbedding(random_state=seedNum, n_components=60, n_jobs=-1),
          'Loc. Lin. Emb. (n=100)':  LocallyLinearEmbedding(random_state=seedNum, n_components=100, n_jobs=-1),
          'MDS (n=10)':              MDS(random_state=seedNum, n_components=10, n_jobs=-1),
          'Spec. Emb. (n=20)':       SpectralEmbedding(random_state=seedNum, n_components=20, n_jobs=-1),
          'Spec. Emb. (n=50)':       SpectralEmbedding(random_state=seedNum, n_components=50, n_jobs=-1),
          'Spec. Emb. (n=100)':      SpectralEmbedding(random_state=seedNum, n_components=100, n_jobs=-1),
          't-SNE (rand/p=70)':       TSNE(random_state=seedNum, perplexity=70, init='random'),
          'Isomap (n=20)':           Isomap(n_components=20, n_jobs=-1),
          'Isomap (n=70)':           Isomap(n_components=70, n_jobs=-1)}

# define ML regressor for reference
MLbase=SGDRegressor(random_state=seedNum, max_iter=iterVal, tol=toleVal)
#MLbase=LassoCV(random_state=seedNum, max_iter=iterVal, tol=toleVal)

print('Dictionary creation complete!')

Each method in the dictionary will be iterated through, and each iteration of stochastic gradient descent will output a mean absolute error for house cost predictions.

In [ ]:
numxVal=10 # number of folds for cross-validation

import time
from sklearn.exceptions import ConvergenceWarning

# initialize dataframe to store ML parameters
resultLabel=(['Pipeline', 'Abs RMS Log Error', 'Training Time (s)'])
results=pd.DataFrame(columns=resultLabel)

# suppress warnings about convergence
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

# first, attempt a round of ML training without applying ANY dim. reduction
t0=time.time()                            # start measuring time to get thru this epoch
MLbase.fit(ds, truth)                     # fit model
MLbase.predict(ds)                        # predict
tTrain=round(time.time()-t0, 1)           # get time needed to calculate
armKa=checkARMSLE(MLbase, ds, truth, armsle, numxVal, 0.3)
resultNow=[['No Reduction', armKa, tTrain]]
results=results.append(pd.DataFrame(resultNow,columns=resultLabel),ignore_index=True)

for redname,rednow in zip(reducers.keys(), reducers.values()):
    t0=time.time()                        # start measuring time to get thru this epoch
    ds_red=rednow.fit_transform(ds)       # apply current transform technique
    MLbase.fit(ds_red, truth)             # fit model
    MLbase.predict(ds_red)                # predict
    tTrain=round(time.time()-t0, 1)       # get time needed to calculate
    armKa=checkARMSLE(MLbase, ds_red, truth, armsle, numxVal, 1)
    resultNow=[[redname, armKa, tTrain]]  # convert result into dataframe
    results=results.append(pd.DataFrame(resultNow,columns=resultLabel),ignore_index=True)
    print('...', end='')                  # just keep loading, just keep loading...

print(); print('Training complete! See below for rough performance result:')
results

These results suggest that cosine-kernel PCA is optimal for this application.

The above learning methods will be trained and cross-validated; this will allow for easy discrimination of the rough performance of the model in the given environment.

### Select Model Type
Next, the best model type and corresponding reduction technique will be identified.

In [ ]:
# import more learning techniques
from sklearn.neighbors      import KNeighborsRegressor;      from sklearn.svm  import SVR
from sklearn.neural_network import MLPRegressor;             from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model   import ElasticNet,Ridge,Lasso

In [ ]:
#reducer={'Loc. Lin. Emb. (n=20)': LocallyLinearEmbedding(random_state=seedNum, n_components=20, n_jobs=-1),
#         'Loc. Lin. Emb. (n=21)': LocallyLinearEmbedding(random_state=seedNum, n_components=21, n_jobs=-1),
#         'Loc. Lin. Emb. (n=22)': LocallyLinearEmbedding(random_state=seedNum, n_components=22, n_jobs=-1)}
reducer=KernelPCA(random_state=seedNum, kernel='cosine', n_components=80, n_jobs=-1)

models={'Ridge Regressor':       Ridge(random_state=seedNum),
        'Elastic Net':           ElasticNet(random_state=seedNum),
        'Lasso Regressor':       Lasso(random_state=seedNum),
        'Decision Tree':         DecisionTreeRegressor(random_state=seedNum),
        'Support Vector Machine':SVR(max_iter=iterVal,tol=toleVal,gamma='scale'),
        'k-Nearest Neighbors':   KNeighborsRegressor(),
        'Fully-Connect. N. Net.':MLPRegressor(random_state=seedNum,solver='lbfgs')}

print('Dictionary creation complete!')

In [ ]:
numxVal=10 # number of folds for cross-validation

resultLabel=(['Reducer','Pipeline','RMS Log Error','Training Time (s)'])
results=pd.DataFrame(columns=resultLabel)     # initialize storage dataframe, again

for MLname,MLnow in zip(models.keys(),  models.values()):
    noRedDone=False                       # reset flag to trigger no-preprocessing analysis
        
    if noRedDone is False:
        noRedDone=True                    # set flag to only skip dim. reduc. once per model type
        t0=time.time()                    # start measuring time to get thru all epochs
        MLnow.fit(ds_red, truth)
        MLnow.predict(ds_red)
        tTrain=round(time.time()-t0, 2)   # get time needed to calculate
        armKa=checkARMSLE(MLnow, ds_red, truth, armsle, numxVal, 0.3)
        resultNow=[['No Reduction', MLname, armKa, tTrain]]  # convert result into dataframe
        results=results.append(pd.DataFrame(resultNow,columns=resultLabel),ignore_index=True)

    t0=time.time()                        # start measuring time to get thru all epochs
    ds_red=reducer.fit_transform(ds)      # apply current transform technique
    MLnow.fit(ds_red, truth)              # fit model
    MLnow.predict(ds_red)                 # predict
    tTrain=round(time.time()-t0, 2)       # get time needed to calculate
    armKa=checkARMSLE(MLnow, ds_red, truth, armsle, numxVal, 0.3)

    resultNow=[['Cosine kPCA', MLname, armKa, tTrain]]  # convert result into dataframe
    results=results.append(pd.DataFrame(resultNow,columns=resultLabel),ignore_index=True)
    print('..', end='')                  # just keep loading, just keep loading...
        
        
print(); print('Training complete! See below for rough performance result:')
results

Given these results, cosine-kernel PCA combined with algorithms other than linear regressors (i.e. not lasso/ridge/elastic net) appears to give the lowest mean absolute error in housing price estimates.

The distribution of inputted features using this manifold transformer is illustrated below.

In [ ]:
from pandas.plotting import scatter_matrix

reducer=KernelPCA(random_state=seedNum, kernel='cosine', n_components=80, n_jobs=-1)
ds_red=reducer.fit_transform(ds)
ds_red=pd.DataFrame(ds_red, index=ds.index) # makee reduced dataset into dataframe

#sctr=scatter_matrix(ds_red, figsize=(12,12), range_padding=0.6, diagonal='kde')
ax=ds_red.plot.density(figsize=(12,3), alpha=0.4, legend=False, xlim=(-0.2, 0.2))

However, the optimal regression algorithm appears to be inconclusive from an accuracy perspective. In the interest of avoiding algorithms that are prone to overfitting, only the support vector machine, k-nearest neighbors, and perceptron approaches will be kept for further analysis.

## Optimize training
The hyperparameters of the above functions will be tuned via grid-search cross-validation.

In [ ]:
models={'k-Nearest Neighbors': KNeighborsRegressor(),
        'Support Vector Mch.': SVR()}

params=[{'n_neighbors':       [3, 5, 7, 10, 15, 20],
         'algorithm':         ['ball_tree','kd_tree'],
         'leaf_size':         [10, 20, 30, 40],
         'weights':           ['uniform','distance']},
        {'kernel':            ['poly','sigmoid','rbf'],
         'gamma':             [0.01,0.05,0.1,0.2,'scale'],
         'epsilon':           [0.1, 0.05, 0.01]},]

print('Regression optimization setup complete!')

In [ ]:
from sklearn.model_selection import GridSearchCV

numxVal=10 # number of folds for cross-validation

modelID=0
for MLname,MLnow in zip(models.keys(),  models.values()):
    paramNow=params[modelID]              # get parameters for current model
    
    ds_red=reducer.fit_transform(ds)      # reduce dimensions
    searchMe=GridSearchCV(MLnow, param_grid=paramNow, cv=numxVal, scoring=armsle, return_train_score=True)
    
    print('Testing',MLname,'...')
    
    searchMe.fit(ds_red, truth)           # try current set of hyperparameters
    error_avg=abs(searchMe.cv_results_['mean_test_score'].mean())
    error_std=np.sqrt(sum(searchMe.cv_results_['std_test_score']**2))
    
    modelID=modelID+1                     # manually iterate to prepare for next model
    
    print('Best Parameters for',MLname,':',searchMe.best_params_)
    print("Abs RMS log error =", round(error_avg,2), '±', round(error_std,2))
    print()

In [ ]:
print('Analysis of perceptrons skipped due to time constraints')
#params={'hidden_layer_sizes':[(30,),(50,),(70),(100)],
#        'activation':        ['tanh','relu'],
#        'solver':            ['lbfgs','sgd','adam'],
#        'learning_rate':     ['constant','invscaling']}

# test perceptron layer dimensions separately, in thee interest of time
#NN=MLPRegressor(random_state=seedNum)

#t0=time.time()                        # start measuring time to get thru all epochs

#for epoch in range(epochs):
#    ds_red=reducer.fit_transform(ds)  # reduce dimensions
#    searchMe=GridSearchCV(NN, param_grid=params, cv=numxVal, scoring=armsle, return_train_score=True)
#    searchMe.fit(ds_red, truth)       # try current set of hyperparameters

#error_avg=abs(searchMe.cv_results_['mean_test_score'].mean())
#error_std=np.sqrt(sum(searchMe.cv_results_['std_test_score']**2))

#print('Best Parameters for',MLname,':',searchMe.best_params_)
#print("Abs RMS log error =", round(error_avg,2), '±', round(error_std,2))
#print()

# Apply desired model to test dataset
Define an ML model and its hyperparameters...

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.manifold  import LocallyLinearEmbedding

epochs=20  # number of epochs to train

reducer=KernelPCA(random_state=seedNum, kernel='cosine', n_components=80, n_jobs=-1)

MLfinal=KNeighborsRegressor(algorithm='ball_tree', leaf_size=10, n_neighbors=5, weights='distance')

for epoch in range(epochs):
    print('Training epoch',epoch+1,'of',epochs)
    ds_red=reducer.fit_transform(ds) # reduce dimensions
    MLfinal.fit(ds_red,truth)        # fit models

...and apply it to the test dataset.

In [ ]:
dtest=pd.read_csv('../Datasets/housePrices/test.csv')

dtest.set_index('Id', inplace=True)              # make entry ID into dataframe index

dt=organize(dtest)                               # encode test dataset
dt_scaled=scaler.transform(dt.values)            # apply robust scalar (see training section)
dt_sca=pd.DataFrame(dt_scaled, index=dt.index, columns=dt.columns)
dt_red=reducer.transform(dt_sca)                 # reduce dimensions

dtest_out=MLfinal.predict(dt_red)                # apply desired ML model to test dataset

dtest_out=np.exp(dtest_out)                      # undo log-normalization of output
dtest_out=pd.DataFrame(dtest_out, index=dt.index, columns=['BaseVal']) # re-format as dataframe
dtest_adj=org_considerMisc(dtest_out, dtest)     # add miscellaneous features' dollar values
dtest_adj.to_csv('test-output.csv', header=True) # save result